In [1]:
import os, os.path
import pandas as pd
import numpy
from sklearn.metrics import log_loss
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform
from sklearn.metrics import log_loss
from sklearn.model_selection import cross_val_score
def my_log_loss(estimator,X,y):
    lloss=log_loss(y,estimator.predict_proba(X))
    return lloss
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

def save_ensemble (name,test_pred,model,df_test_listingID):
    label_dictionary ={1 : 'low',  2:'medium', 3:'high' } 
    columns=[label_dictionary[key] for key in model.classes_]
    columns_out=['listing_id', 'high', 'medium', 'low']
    predictions=pd.DataFrame(test_pred,columns=columns)
    predictions['listing_id']=df_test_listingID.reset_index(drop=True)
    predictions=predictions[columns_out]
    predictions.to_csv('ensemble_predictions/'+name+'.csv', index=False)


def weighted_mean(files,weights,dir="Thesis_predictions/"):
    columns=['high','medium','low']
    file_init=files[0]
    df=pd.read_csv(dir+file_init)
    df[columns]*=weights[0]

    for file,weight in zip(files[1:],weights[1:]):
        #print(weight,file)
        df_new=pd.read_csv(dir+file)
        df[columns]+=weight*df_new[columns]
        
    df[columns]/=sum(weights)
    return df

def summary(y,yoof_files,test_files,yoof_weights,test_weights):
    yoof=weighted_mean(yoof_files,yoof_weights)
    lloss=log_loss(y,yoof[['low','medium','high']])   
    print(lloss)
    return weighted_mean(test_files,test_weights),lloss
def get_rank(weights):  
    n=len(weights)
    rank=numpy.argsort(weights)
    new_weights=[0]*n
    for i in range(n):
        new_weights[rank[i]]=i+1
    return new_weights
def save_ensemble_avg (name,df,lloss):
    df.to_csv('ensemble_predictions/'+name+'_'+lloss+'.csv', index=False)

In [24]:

yoof_files=[name for name in os.listdir(DIR) if 'yoof' in name and os.path.isfile(os.path.join(DIR, name))]
len(yoof_files)

546

In [11]:
DIR = 'Thesis_predictions/'
df_train= pd.read_json('train.json')
df_test= pd.read_json('test.json')
event_dictionary ={'low' : 1, 'medium' : 2, 'high' : 3} 
df_train['interest_integer'] =df_train['interest_level'].map(event_dictionary) 
df=df_train.drop(df_train.columns.difference(['listing_id','interest_integer']), 1).reset_index(drop=True)
df_pred=df_test.drop(df_test.columns.difference(['listing_id']), 1).reset_index(drop=True)
#skf = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)
X=df.iloc[:,2:]
y=df.iloc[:,1]
X_test=df_pred.iloc[:,1:]



test_files=[name for name in os.listdir(DIR) if 'yoof' not in name and os.path.isfile(os.path.join(DIR, name))]
yoof_weights=[1-float(file.split("_yoof")[0].split('_')[2])  if 'log' not in file else 1-float(file.split("log")[0].split('_')[2]) for file in yoof_files]
test_weights=[1-float(file.split(".csv")[0].split('_')[2])  if 'log' not in file  else 1-float(file.split("log")[0].split('_')[2]) for file in test_files]

In [2]:
def parse_score(file):
    if 'log' in file:
        return float(file.split("log")[0].split('_')[2]) 
    return float(file.split("_yoof")[0].split('_')[2])
    

In [6]:
def parse_score_1(file):
    file=file.replace('log','').replace('_yoof','').replace('_elastic_net','').replace('.csv','').split('_')[-1]
    return float(file)


In [3]:
DIR = 'Thesis_predictions/'

In [4]:
SCORE_TRESH=1

In [13]:
yoof_files=[name for name in os.listdir(DIR) if 'yoof' in name and os.path.isfile(os.path.join(DIR, name))]
yoof_weights=[parse_score_1(file) for file in yoof_files]
yoof_weights_index=[i  for i,score in enumerate(yoof_weights) if score<SCORE_TRESH]
yoof_files=[yoof_files[i] for i in yoof_weights_index]
yoof_weights=[1-yoof_weights[i] for i in yoof_weights_index]

In [14]:
test_files=[name for name in os.listdir(DIR) if 'yoof' not in name and os.path.isfile(os.path.join(DIR, name))]
test_weights=[parse_score_1(file) for file in test_files]
test_weights_index=[i  for i,score in enumerate(test_weights) if score<SCORE_TRESH]
test_files=[test_files[i] for i in test_weights_index]
test_weights=[1-test_weights[i] for i in test_weights_index]

In [15]:
pd.DataFrame({'a':test_weights,'b':test_files})

,a,b
0,0.389,knn2Pipeline_047bf4_0.611.csv
1,0.391,knn2Pipeline_10aa15_0.609.csv
2,0.374,knn2Pipeline_12e3f6_0.626.csv
3,0.389,knn2Pipeline_19e32f_0.611.csv
4,0.389,knn2Pipeline_2086f5_0.611.csv
...,...,...
515,0.451,XGBClassifier_f8920e_0.549.csv
516,0.456,XGBClassifier_f8a40e_0.544.csv
517,0.318,XGBClassifier_fbfe02_0.682.csv
518,0.463,XGBClassifier_fd8ed3_0.537.csv


In [52]:
yoof_weights==test_weights

True

## Average of predictions

In [186]:
df,lloss=summary(y,yoof_files,test_files,[1]*len(yoof_files),[1]*len(test_files))

0.5447827239567172


In [16]:
df,lloss=summary(y,yoof_files,test_files,[1]*len(yoof_files),[1]*len(test_files))
#save_ensemble_avg ('average<1',df,str(lloss)) 

#0.55497  <1

0.5517133514762225


In [187]:
df,lloss=summary(y,yoof_files,test_files,yoof_weights,test_weights)

0.5441216952642852


In [17]:
df,lloss=summary(y,yoof_files,test_files,yoof_weights,test_weights)
#save_ensemble_avg ('weighted_average<1',df,str(lloss)) 

#0.55381 <1

0.5503712722410666


In [188]:
yoof_rank_w=get_rank(yoof_weights)
test_rank_w=get_rank(test_weights)
df,lloss=summary(y,yoof_files,test_files,yoof_rank_w,test_rank_w)

0.5366607399805968


In [18]:
yoof_rank_w=get_rank(yoof_weights)
test_rank_w=get_rank(test_weights)
df,lloss=summary(y,yoof_files,test_files,yoof_rank_w,test_rank_w)
#save_ensemble_avg ('weighted_rank<1',df,str(lloss)) 
#0.54557 <1   53995

0.5410502637302053


In [393]:
df,lloss=summary(y,yoof_files,test_files,[1]*len(yoof_files),[1]*len(test_files))
#save_ensemble_avg ('average',df,str(lloss)) 

0.5368196341518756


In [394]:
df,lloss=summary(y,yoof_files,test_files,yoof_weights,test_weights)
save_ensemble_avg ('weighted_average',df,str(lloss)) 

0.536451449645222


In [395]:
yoof_rank_w=get_rank(yoof_weights)
test_rank_w=get_rank(test_weights)
df,lloss=summary(y,yoof_files,test_files,yoof_rank_w,test_rank_w)
save_ensemble_avg ('weighted_rank',df,str(lloss)) 

0.535244807162214


In [22]:
d = {'estimated log loss': [0.55110, 0.54982,0.54067,0.52518], 'leaderboard log loss': [0.55625,0.55504,0.54643,0.53189]}
df = pd.DataFrame(data=d,index=['Average model','Weighted average model','Rank weighted model','Stacking with LGBM'])
df

,estimated log loss,leaderboard log loss
Average model,0.55110,0.55625
Weighted average model,0.54982,0.55504
Rank weighted model,0.54067,0.54643
Stacking with LGBM,0.52518,0.53032


## Trenovanie modelov

In [247]:
DIR = 'Thesis_predictions/'
files=[name for name in os.listdir(DIR) if  os.path.isfile(os.path.join(DIR, name))]
df=pd.DataFrame()
df_pred=pd.DataFrame()
for file in yoof_files:
    if 'yoof' in file:      
        yoof_pred= pd.read_csv(DIR+file)
        rsuffix='_'+file.split("_yoof")[0]
        for column in yoof_pred.columns:
            if column=='listing_id':
                continue
            else: 
                df[column+rsuffix]=yoof_pred[column]
for file in test_files:
    predictions= pd.read_csv(DIR+file)
    rsuffix='_'+file.split(".csv")[0]
    for column in predictions.columns:
        if column=='listing_id':
            continue
        else: 
            df_pred[column+rsuffix]=predictions[column]
        


In [239]:
set(df.columns)-set(df_pred.columns),set(df_pred.columns)-set(df.columns)

(set(), set())

In [248]:
event_dictionary ={'low' : 1, 'medium' : 2, 'high' : 3} 
df_train= pd.read_json('train.json')
df_test= pd.read_json('test.json')
target=df_train.set_index('listing_id')['interest_level'].map(event_dictionary)
df['listing_id']=df_train['listing_id'].reset_index(drop=True)
df_pred['listing_id']=df_test['listing_id'].reset_index(drop=True)
df=df.set_index('listing_id')
df_pred=df_pred.set_index('listing_id')
df['training']=1
df_pred['training']=0
X=pd.concat([df,df_pred])
X['target']=target

In [190]:
from modeling import model_predict

def model_predict2(df,training_mask,target, clf = RandomForestClassifier(max_depth=5, random_state=0,n_estimators=100,n_jobs=-1),encoder_dict={},n_splits=5):   
    X=df[training_mask==1]
    X_test=df[training_mask==0]
    y=target[training_mask==1]
    skf = StratifiedKFold(n_splits=n_splits, random_state=42, shuffle=True)
    scores=list()
    yoof=np.zeros((X.shape[0],3))
    test_pred=np.zeros((X_test.shape[0],3))

            
    for train, val in skf.split(X, y):
        X_train=X.iloc[train]
        y_train=y.iloc[train]
        X_val=X.iloc[val]
        y_val=y.iloc[val]
        
        ##
        predict_enc(encoder_dict,X_train,X_val,y_train)
        clf.fit(X_train.drop(columns=encoder_dict.keys()), y_train)
        preds=clf.predict_proba(X_val.drop(columns=encoder_dict.keys()))
        scores.append(log_loss(y_val, preds))
        yoof[val]=preds
    X,X_test=predict_enc(encoder_dict,X,X_test,y)
    clf.fit(X.drop(columns=encoder_dict.keys()), y)
    test_pred=clf.predict_proba(X_test.drop(columns=encoder_dict.keys()))
    return test_pred,yoof,log_loss(y,yoof)

In [200]:
from lightgbm import LGBMClassifier
clf=LGBMClassifier(n_jobs=-1,n_estimators=100,random_state=0)
test_pred,yoof,log_loss_score=model_predict(X.drop(columns=['target','training']),X['training'],X['target'], clf = clf)
log_loss_score

0.5298522298766846

In [142]:
log_loss_score

0.5287512020420081

In [122]:
log_loss_score

0.5297952331156376

In [199]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(max_depth=5,criterion='entropy', random_state=0,n_estimators=150,n_jobs=-1)
test_pred,yoof,log_loss_score=model_predict(X.drop(columns=['target','training']),X['training'],X['target'], clf = clf)
log_loss_score

0.5274414913577815

In [147]:
log_loss_score
save_ensemble('ensemble_RF',test_pred,clf,df_test['listing_id'])

In [107]:
save_ensemble('ensemble_lgbm_0.5287',test_pred,clf,df_test['listing_id'])

In [202]:
import xgboost as xgb
clf=xgb.XGBClassifier(n_jobs=-1,random_state=0,tree_method='gpu_hist')
test_pred,yoof,log_loss_score=model_predict(X.drop(columns=['target','training']),X['training'],X['target'], clf = clf)

In [203]:
log_loss_score

0.5492681461568192

In [249]:
X

,high_XGBClassifier_b7bffc_0.537,medium_XGBClassifier_b7bffc_0.537,low_XGBClassifier_b7bffc_0.537,high_LGBMClassifier_a2f6d9_0.538,medium_LGBMClassifier_a2f6d9_0.538,low_LGBMClassifier_a2f6d9_0.538,high_LGBMClassifier_4265b5_0.537,medium_LGBMClassifier_4265b5_0.537,low_LGBMClassifier_4265b5_0.537,high_XGBClassifier_f8920e_0.549,...,medium_LGBMClassifier_4dfd2f_0.543,low_LGBMClassifier_4dfd2f_0.543,high_XGBClassifier_f5884f_0.536,medium_XGBClassifier_f5884f_0.536,low_XGBClassifier_f5884f_0.536,high_XGBClassifier_570512_0.547,medium_XGBClassifier_570512_0.547,low_XGBClassifier_570512_0.547,training,target
listing_id,,,,,,,,,,,,,,,,,,,,,
7170325,0.015991,0.383648,0.600361,0.014799,0.288568,0.696633,0.024960,0.374595,0.600445,0.050439,...,0.342669,0.646591,0.025680,0.419893,0.554427,0.032783,0.403438,0.563779,1,2.000000
7092344,0.076603,0.442511,0.480887,0.082838,0.589275,0.327887,0.062741,0.622731,0.314528,0.078833,...,0.662088,0.295039,0.069985,0.462028,0.467987,0.077435,0.355732,0.566833,1,1.000000
7158677,0.012489,0.201786,0.785725,0.009752,0.300312,0.689937,0.012250,0.250385,0.737365,0.022295,...,0.274707,0.718715,0.014403,0.235764,0.749833,0.023209,0.226445,0.750346,1,2.000000
7211212,0.031132,0.229040,0.739828,0.020930,0.199562,0.779508,0.036899,0.255091,0.708011,0.056861,...,0.244794,0.736652,0.040647,0.292849,0.666504,0.041665,0.366000,0.592334,1,2.000000
7225292,0.001184,0.055771,0.943044,0.002251,0.066077,0.931672,0.003725,0.064699,0.931577,0.006041,...,0.052655,0.945838,0.001744,0.046608,0.951648,0.001680,0.069370,0.928950,1,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6928108,0.660341,0.223336,0.116323,0.625436,0.248864,0.125700,0.466859,0.379503,0.153639,0.450029,...,0.294405,0.167000,0.619873,0.256706,0.123421,0.652127,0.261873,0.086000,0,nan
6906674,0.009475,0.257527,0.732999,0.007384,0.218222,0.774394,0.005954,0.214454,0.779591,0.012444,...,0.245726,0.749300,0.007982,0.233083,0.758935,0.011999,0.247545,0.740456,0,nan
6897967,0.000316,0.014360,0.985324,0.000709,0.024006,0.975284,0.001355,0.026357,0.972288,0.004930,...,0.016511,0.982933,0.000590,0.015106,0.984304,0.000654,0.022459,0.976887,0,nan


In [219]:
clf=LogisticRegression(penalty= 'l2', C = 0.1 ,solver='saga', tol=1e-2, max_iter=200,random_state=0)
test_pred,yoof,log_loss_score=model_predict(X.drop(columns=['target','training']),X['training'],X['target'], clf = clf)


In [220]:
log_loss_score #L2 0.1

0.5346526982370798

In [209]:
log_loss_score #L2 0.1

0.5346526982370798

In [205]:
log_loss_score #L1 0.1

0.5370716937496343

In [241]:
X

,high_RandomForestClassifier_b4dd0f_0.579,medium_RandomForestClassifier_b4dd0f_0.579,low_RandomForestClassifier_b4dd0f_0.579,high_XGBClassifier_b7bffc_0.537,medium_XGBClassifier_b7bffc_0.537,low_XGBClassifier_b7bffc_0.537,high_LGBMClassifier_a2f6d9_0.538,medium_LGBMClassifier_a2f6d9_0.538,low_LGBMClassifier_a2f6d9_0.538,high_LGBMClassifier_4265b5_0.537,...,medium_XGBClassifier_f5884f_0.536,low_XGBClassifier_f5884f_0.536,high_XGBClassifier_570512_0.547,medium_XGBClassifier_570512_0.547,low_XGBClassifier_570512_0.547,high_RandomForestClassifier_e597d7_0.569,medium_RandomForestClassifier_e597d7_0.569,low_RandomForestClassifier_e597d7_0.569,training,target
listing_id,,,,,,,,,,,,,,,,,,,,,
7170325,0.097993,0.419553,0.482454,0.015991,0.383648,0.600361,0.014799,0.288568,0.696633,0.024960,...,0.419893,0.554427,0.032783,0.403438,0.563779,0.089511,0.451286,0.459203,1,2.000000
7092344,0.087893,0.397851,0.514256,0.076603,0.442511,0.480887,0.082838,0.589275,0.327887,0.062741,...,0.462028,0.467987,0.077435,0.355732,0.566833,0.080648,0.420899,0.498453,1,1.000000
7158677,0.022211,0.239795,0.737994,0.012489,0.201786,0.785725,0.009752,0.300312,0.689937,0.012250,...,0.235764,0.749833,0.023209,0.226445,0.750346,0.015490,0.244513,0.739997,1,2.000000
7211212,0.049631,0.255677,0.694692,0.031132,0.229040,0.739828,0.020930,0.199562,0.779508,0.036899,...,0.292849,0.666504,0.041665,0.366000,0.592334,0.048544,0.248181,0.703275,1,2.000000
7225292,0.005010,0.074486,0.920504,0.001184,0.055771,0.943044,0.002251,0.066077,0.931672,0.003725,...,0.046608,0.951648,0.001680,0.069370,0.928950,0.001870,0.049536,0.948594,1,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6928108,0.389980,0.407730,0.202289,0.660341,0.223336,0.116323,0.625436,0.248864,0.125700,0.466859,...,0.256706,0.123421,0.652127,0.261873,0.086000,0.375048,0.439519,0.185433,0,nan
6906674,0.014163,0.172779,0.813058,0.009475,0.257527,0.732999,0.007384,0.218222,0.774394,0.005954,...,0.233083,0.758935,0.011999,0.247545,0.740456,0.009236,0.164934,0.825830,0,nan
6897967,0.013008,0.078832,0.908160,0.000316,0.014360,0.985324,0.000709,0.024006,0.975284,0.001355,...,0.015106,0.984304,0.000654,0.022459,0.976887,0.007314,0.048164,0.944522,0,nan


In [143]:
clf=LogisticRegression(random_state=0,solver='lbfgs', C = 1e-2 ,multi_class='auto',n_jobs=-1,fit_intercept=True,max_iter=100)
test_pred,yoof,log_loss_score=model_predict(X.drop(columns=['target','training']),X['training'],X['target'], clf = clf)
log_loss_score

0.5352596213773132

In [19]:
clf = LogisticRegression(random_state=0)
test_pred,yoof,log_loss_score=model_predict(X.drop(columns=['target','training']),X['training'],X['target'], clf = clf)

/home/martas/anaconda3/envs/mirko/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/home/martas/anaconda3/envs/mirko/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/

In [20]:
log_loss_score

0.5360029002203738

In [23]:
save_ensemble('ensemble_logistic',test_pred,clf,df_test['listing_id'])

In [ ]:
test_pred,yoof,log_loss_score

In [250]:
X.to_csv('ensemble_predictions_v1.csv', index=True)